<a href="https://colab.research.google.com/github/AwaisEhsan67/Kyber512-and-NTRU509-comparison/blob/main/PQClean_Library_for_Kyber_512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get update
!sudo apt-get install build-essential git


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,545 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,208 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd

In [2]:
!git clone https://github.com/PQClean/PQClean.git


Cloning into 'PQClean'...
remote: Enumerating objects: 33664, done.
remote: Counting objects: 100% (3101/3101), done.
remote: Compressing objects: 100% (724/724), done.
remote: Total 33664 (delta 2770), reused 2377 (delta 2377), pack-reused 30563 (from 3)
Receiving objects: 100% (33664/33664), 7.43 MiB | 20.96 MiB/s, done.
Resolving deltas: 100% (26181/26181), done.


In [13]:
cd PQClean/crypto_kem/ml-kem-512/clean/


/content/PQClean/crypto_kem/ml-kem-512/clean


In [21]:
code = """
#include <stdio.h>
#include <stdint.h>
#include <stdlib.h>
#include <string.h>
#include "api.h"

int main() {
    uint8_t pk[PQCLEAN_MLKEM512_CLEAN_CRYPTO_PUBLICKEYBYTES];
    uint8_t sk[PQCLEAN_MLKEM512_CLEAN_CRYPTO_SECRETKEYBYTES];
    uint8_t ct[PQCLEAN_MLKEM512_CLEAN_CRYPTO_CIPHERTEXTBYTES];
    uint8_t key_a[PQCLEAN_MLKEM512_CLEAN_CRYPTO_BYTES];
    uint8_t key_b[PQCLEAN_MLKEM512_CLEAN_CRYPTO_BYTES];

    PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair(pk, sk);
    PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc(ct, key_a, pk);
    PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec(key_b, ct, sk);

    printf("Shared keys match: %s\\n", memcmp(key_a, key_b, PQCLEAN_MLKEM512_CLEAN_CRYPTO_BYTES) == 0 ? "YES" : "NO");

    return 0;
}
"""
with open("test_kyber.c", "w") as f:
    f.write(code)


In [22]:
!ls


api.h	  kem.c			    ntt.c     polyvec.c    symmetric-shake.c
cbd.c	  kem.h			    ntt.h     polyvec.h    test_kyber.c
cbd.h	  LICENSE		    params.h  reduce.c	   verify.c
indcpa.c  Makefile		    poly.c    reduce.h	   verify.h
indcpa.h  Makefile.Microsoft_nmake  poly.h    symmetric.h


In [23]:
!ls test_kyber.c
!ls kem.c indcpa.c verify.c cbd.c ntt.c reduce.c polyvec.c poly.c symmetric-shake.c
!ls ../../../common/randombytes.c
!ls ../../../common/fips202.c


test_kyber.c
cbd.c	  kem.c  poly.c     reduce.c	       verify.c
indcpa.c  ntt.c  polyvec.c  symmetric-shake.c
../../../common/randombytes.c
../../../common/fips202.c


In [24]:
!gcc -o test_kyber test_kyber.c \
    kem.c indcpa.c verify.c cbd.c ntt.c reduce.c polyvec.c poly.c symmetric-shake.c \
    ../../../common/randombytes.c \
    ../../../common/fips202.c \
    -I . -I ../../../common


In [25]:
!./test_kyber


Shared keys match: YES


In [26]:
!gcc -shared -fPIC \
    kem.c indcpa.c verify.c cbd.c ntt.c reduce.c polyvec.c poly.c symmetric-shake.c \
    ../../../common/randombytes.c ../../../common/fips202.c \
    -I . -I ../../../common \
    -o libkyber512.so


In [27]:
import ctypes
import os


lib = ctypes.CDLL("./libkyber512.so")


pk = ctypes.create_string_buffer(800)
sk = ctypes.create_string_buffer(1632)
ct = ctypes.create_string_buffer(768)
ss = ctypes.create_string_buffer(32)


lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]


lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair(pk, sk)
lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc(ct, ss, pk)
ss_dec = ctypes.create_string_buffer(32)
lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec(ss_dec, ct, sk)


print("Shared secret match:", ss.raw == ss_dec.raw)


Shared secret match: True


In [28]:
import ctypes
import time
import statistics


lib = ctypes.CDLL("./libkyber512.so")


lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair.argtypes = [ctypes.c_char_p, ctypes.c_char_p]


CRYPTO_PUBLICKEYBYTES = 800
CRYPTO_SECRETKEYBYTES = 1632


trials = 1000
times = []

for _ in range(trials):
    pk = ctypes.create_string_buffer(CRYPTO_PUBLICKEYBYTES)
    sk = ctypes.create_string_buffer(CRYPTO_SECRETKEYBYTES)

    start = time.perf_counter()
    lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair(pk, sk)
    end = time.perf_counter()

    times.append((end - start) * 1000)  # ms


print(f"Ran {trials} trials of Kyber-512 key generation")
print(f"Average time: {statistics.mean(times):.3f} ms")
print(f"Min time: {min(times):.3f} ms")
print(f"Max time: {max(times):.3f} ms")
print(f"Standard deviation: {statistics.stdev(times):.3f} ms")


Ran 1000 trials of Kyber-512 key generation
Average time: 0.163 ms
Min time: 0.155 ms
Max time: 0.665 ms
Standard deviation: 0.025 ms


In [30]:
import ctypes
import time
import statistics

lib = ctypes.CDLL("./libkyber512.so")

KYBER_PUBLICKEYBYTES  = 800
KYBER_SECRETKEYBYTES  = 1632
KYBER_CIPHERTEXTBYTES = 768
KYBER_SSBYTES         = 32


keypair = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
keypair.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
keypair.restype  = ctypes.c_int

encapsulate = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc
encapsulate.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
encapsulate.restype  = ctypes.c_int

decapsulate = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec
decapsulate.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
decapsulate.restype  = ctypes.c_int

def functional_test():
    pk = ctypes.create_string_buffer(KYBER_PUBLICKEYBYTES)
    sk = ctypes.create_string_buffer(KYBER_SECRETKEYBYTES)
    ct = ctypes.create_string_buffer(KYBER_CIPHERTEXTBYTES)
    ss = ctypes.create_string_buffer(KYBER_SSBYTES)
    ss_dec = ctypes.create_string_buffer(KYBER_SSBYTES)

    assert keypair(pk, sk) == 0, "Keypair failed"
    assert encapsulate(ct, ss, pk) == 0, "Encapsulation failed"
    assert decapsulate(ss_dec, ct, sk) == 0, "Decapsulation failed"

    match = ss.raw == ss_dec.raw
    print(f"[Functional] Shared secret match: {match}")

def benchmark(fn, buf_sizes, trials=1000):
    """Run fn(bufs...) for trials and return list of ms timings."""
    times = []
    for _ in range(trials):
        bufs = [ctypes.create_string_buffer(sz) for sz in buf_sizes]
        start = time.perf_counter()
        assert fn(*bufs) == 0
        end = time.perf_counter()
        times.append((end - start) * 1000)
    return times

def print_stats(name, times):
    print(f"[Benchmark] {name}:")
    print(f"  avg = {statistics.mean(times):.3f} ms")
    print(f"  min = {min(times):.3f} ms")
    print(f"  max = {max(times):.3f} ms")
    print(f"  std = {statistics.stdev(times):.3f} ms")


if __name__ == "__main__":

    functional_test()


    trials = 1000

    t_keygen = benchmark(keypair, [KYBER_PUBLICKEYBYTES, KYBER_SECRETKEYBYTES], trials)
    print_stats("Kyber KeyGen", t_keygen)


    t_enc = benchmark(encapsulate,
                      [KYBER_CIPHERTEXTBYTES, KYBER_SSBYTES, KYBER_PUBLICKEYBYTES],
                      trials)
    print_stats("Kyber Encapsulate", t_enc)


    t_dec = benchmark(decapsulate,
                      [KYBER_SSBYTES, KYBER_CIPHERTEXTBYTES, KYBER_SECRETKEYBYTES],
                      trials)
    print_stats("Kyber Decapsulate", t_dec)


[Functional] Shared secret match: True
[Benchmark] Kyber KeyGen:
  avg = 0.163 ms
  min = 0.155 ms
  max = 0.437 ms
  std = 0.018 ms
[Benchmark] Kyber Encapsulate:
  avg = 0.209 ms
  min = 0.201 ms
  max = 0.590 ms
  std = 0.023 ms
[Benchmark] Kyber Decapsulate:
  avg = 0.279 ms
  min = 0.265 ms
  max = 0.764 ms
  std = 0.036 ms


In [31]:
!pip install psutil


In [8]:
!sudo apt-get update
!sudo apt-get install time

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,208 kB]
Fetched 3,593 kB in 2s (2,360 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubunt

In [9]:
!which time

/usr/bin/time


In [14]:
import ctypes
import psutil
import time


lib = ctypes.CDLL("./libkyber512.so")

keypair = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
keypair.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
keypair.restype  = ctypes.c_int


KYBER_PK = 800
KYBER_SK = 1632


proc = psutil.Process()


pk = ctypes.create_string_buffer(KYBER_PK)
sk = ctypes.create_string_buffer(KYBER_SK)
keypair(pk, sk)

# Force a short sleep to let OS settle memory
time.sleep(0.1)


mem_before = proc.memory_info().rss


pk = ctypes.create_string_buffer(KYBER_PK)
sk = ctypes.create_string_buffer(KYBER_SK)
keypair(pk, sk)

mem_after = proc.memory_info().rss

print(f"Kyber-512 KeyGen Memory (RSS):")
print(f"  Before: {mem_before/1024:.1f} KB")
print(f"  After:  {mem_after/1024:.1f} KB")
print(f"  Delta:  {(mem_after-mem_before)/1024:.1f} KB")


Kyber-512 KeyGen Memory (RSS):
  Before: 103084.0 KB
  After:  103084.0 KB
  Delta:  0.0 KB


In [15]:

%%shell
/usr/bin/time -f "Kyber-512 KeyGen PeakRSS: %M KB" python3 - << 'PYCODE'
import ctypes

lib = ctypes.CDLL("./libkyber512.so")
kp  = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
kp.argtypes = [ctypes.c_void_p, ctypes.c_void_p]

pk = ctypes.create_string_buffer(800)
sk = ctypes.create_string_buffer(1632)
kp(pk, sk)
PYCODE


Kyber-512 KeyGen PeakRSS: 13576 KB


In [16]:
import ctypes
import time

kyber = ctypes.CDLL("./libkyber512.so")
keypair = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
keypair.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
keypair.restype  = ctypes.c_int

encapsulate = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc
encapsulate.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
encapsulate.restype  = ctypes.c_int

decapsulate = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec
decapsulate.argtypes = [ctypes.c_char_p, ctypes.c_char_p, ctypes.c_char_p]
decapsulate.restype  = ctypes.c_int


pk = ctypes.create_string_buffer(800)
sk = ctypes.create_string_buffer(1600)
ct = ctypes.create_string_buffer(768)
ss = ctypes.create_string_buffer(32)

start = time.perf_counter_ns()
keypair(pk, sk)
keygen_time = time.perf_counter_ns() - start

start = time.perf_counter_ns()
encapsulate(ct, ss, pk)
enc_time = time.perf_counter_ns() - start

start = time.perf_counter_ns()
decapsulate(ss, ct, sk)
dec_time = time.perf_counter_ns() - start


keygen_time //= 1000
enc_time    //= 1000
dec_time    //= 1000

print("KeyGen:", keygen_time, "µs")
print("Encapsulation:", enc_time, "µs")
print("Decapsulation:", dec_time, "µs")


KeyGen: 407 µs
Encapsulation: 443 µs
Decapsulation: 576 µs


In [1]:
!lscpu


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4400.30
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

In [2]:
!free -h


               total        used        free      shared  buff/cache   available
Mem:            12Gi       705Mi       7.2Gi       2.0Mi       4.8Gi        11Gi
Swap:             0B          0B          0B


In [3]:
!cat /proc/cpuinfo
!cat /proc/meminfo


processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2200.150
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4400.30
clflush size	: 64
cache_alignment	: 64
ad

In [4]:
!pip install pyRAPL


In [ ]:

# following code will work on laptop and Raspberry Pi only, not on collab.
import ctypes
import pyRAPL
import sys


pyRAPL.setup()
meter = pyRAPL.Measurement('kyber_full_kem')


lib = ctypes.CDLL("./libkyber512.so")
kp = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_keypair
enc = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_enc
dec = lib.PQCLEAN_MLKEM512_CLEAN_crypto_kem_dec

kp.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
enc.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
dec.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]


PK_BYTES = 800
SK_BYTES = 1632
CT_BYTES = 768
SS_BYTES = 32

pk = ctypes.create_string_buffer(PK_BYTES)
sk = ctypes.create_string_buffer(SK_BYTES)
ct = ctypes.create_string_buffer(CT_BYTES)
ss = ctypes.create_string_buffer(SS_BYTES)


meter.begin()
kp(pk, sk)
enc(ct, ss, pk)
dec(ss, ct, sk)
meter.end()

print("Kyber-512 full KEM energy (µJ):", meter.result.pkg)


In [5]:
import time

bandwidth_mbps = 10  # Mbps
bandwidth_bps = bandwidth_mbps * 1_000_000

kyber_total_bytes = 800 + 768  # pk + ct
ntru_total_bytes = 699 + 699   # pk + ct

def simulate_network_transfer(data_bytes):
    data_bits = data_bytes * 8
    expected_time = data_bits / bandwidth_bps  # seconds
    start = time.time()
    time.sleep(expected_time)
    end = time.time()
    return end - start


kyber_time = simulate_network_transfer(kyber_total_bytes)
print(f"Kyber512 simulated transmission time: {kyber_time * 1000:.2f} ms")


ntru_time = simulate_network_transfer(ntru_total_bytes)
print(f"NTRU509 simulated transmission time: {ntru_time * 1000:.2f} ms")


Kyber512 simulated transmission time: 2.41 ms
NTRU509 simulated transmission time: 1.19 ms
